In [1149]:
!pip install tweepy --quiet

In [1150]:
import sys
sys.executable

'c:\\Users\\14708\\anaconda3\\python.exe'

In [1197]:
import random
import numpy as np 
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 50)


import nltk
from nltk.tokenize import word_tokenize

In [1152]:
main_df=pd.read_csv(r"C:\Users\14708\Downloads\Tata and Hyundai cars - data.xlsx - Data.csv")
df = main_df.copy()

## Data Exploration

In [1153]:
df.head()

,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,Language,Reach,Desktop Reach,Mobile Reach,Twitter Social Echo,Facebook Social Echo,Reddit Social Echo,National Viewership,Engagement,Sentiment,Keywords,State
0,28-Sep-2023 02:08PM,Kia और Hyundai ने वापस बुलाईं अपनी 35 लाख कारे...,https://hindi.pardaphash.com/kia-and-hyundai-r...,Kia and Hyundai Recalled Cars: वाहन निर्माता क...,Kia and Hyundai Recalled Cars: वाहन निर्माता क...,Pardaphash,NaN,India,Delhi,Hindi,31307.0,1279.0,30028.0,0.0,0.0,0.0,0,NaN,Neutral,"Hyundai,Cars",Delhi
1,28-Sep-2023 01:46PM,"Tata Punch EV spied, may compete with MG Comen...",https://economictimes.indiatimes.com/industry/...,Tata Motors is reportedly developing an electr...,... Punch EV is expected to compete with the M...,The Economic Times,NaN,India,National,English,15403059.0,6171732.0,9231327.0,0.0,0.0,0.0,0,NaN,Neutral,"Tata,cars",Maharashtra
2,28-Sep-2023 01:30PM,Tata Avinya Reviews On Road Price 2023 Mileage...,https://badisoch.in/automobile-news/tata-aviny...,... (adsbygoogle = window.adsbygoogle || []).p...,"... , despite most carmakers focusing on large...",Badi Soch,NaN,India,NaN,English,1128280.0,171549.0,956731.0,0.0,0.0,0.0,0,NaN,Neutral,"cars,Tata",NaN
3,28-Sep-2023 01:24PM,NaN,http://twitter.com/automobilindia8/statuses/17...,NaN,Engine Fire-Related Risks Force Kia & Hyundai ...,Twitter,@automobilindia8,India,NaN,English,2.0,NaN,NaN,NaN,NaN,NaN,0,0.0,Neutral,"Hyundai,Cars,hyundai",Karnataka
4,28-Sep-2023 01:22PM,NaN,http://twitter.com/MalakpetD/statuses/17073021...,NaN,"@srinualavilli @BMTC_BENGALURU Mahindra, Tata,...",Twitter,@MalakpetD,India,NaN,English,506.0,NaN,NaN,NaN,NaN,NaN,0,0.0,Neutral,"Tata,cars",Telangana


In [1154]:
df.iloc[0]

Date                                                  28-Sep-2023 02:08PM
Headline                Kia और Hyundai ने वापस बुलाईं अपनी 35 लाख कारे...
URL                     https://hindi.pardaphash.com/kia-and-hyundai-r...
Opening Text            Kia and Hyundai Recalled Cars: वाहन निर्माता क...
Hit Sentence            Kia and Hyundai Recalled Cars: वाहन निर्माता क...
Source                                                         Pardaphash
Influencer                                                            NaN
Country                                                             India
Subregion                                                           Delhi
Language                                                            Hindi
Reach                                                             31307.0
Desktop Reach                                                      1279.0
Mobile Reach                                                      30028.0
Twitter Social Echo                   

In [1155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528 entries, 0 to 1527
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  1528 non-null   object 
 1   Headline              710 non-null    object 
 2   URL                   1528 non-null   object 
 3   Opening Text          710 non-null    object 
 4   Hit Sentence          1528 non-null   object 
 5   Source                1528 non-null   object 
 6   Influencer            1065 non-null   object 
 7   Country               1528 non-null   object 
 8   Subregion             321 non-null    object 
 9   Language              1528 non-null   object 
 10  Reach                 1519 non-null   float64
 11  Desktop Reach         710 non-null    float64
 12  Mobile Reach          710 non-null    float64
 13  Twitter Social Echo   710 non-null    float64
 14  Facebook Social Echo  710 non-null    float64
 15  Reddit Social Echo   

In [1156]:
df.columns

Index(['Date', 'Headline', 'URL', 'Opening Text', 'Hit Sentence', 'Source',
       'Influencer', 'Country', 'Subregion', 'Language', 'Reach',
       'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo',
       'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership',
       'Engagement', 'Sentiment', 'Keywords', 'State'],
      dtype='object')

In [1157]:
categorical_cols = df.select_dtypes('object').columns
numeric_cols = df.select_dtypes(include=np.number).columns

print('categorical_cols:', categorical_cols)
print('numeric_cols:', numeric_cols)

categorical_cols: Index(['Date', 'Headline', 'URL', 'Opening Text', 'Hit Sentence', 'Source',
       'Influencer', 'Country', 'Subregion', 'Language', 'Sentiment',
       'Keywords', 'State'],
      dtype='object')
numeric_cols: Index(['Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo',
       'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership',
       'Engagement'],
      dtype='object')


In [1158]:
# Value counts for categorical columns
def val_count(dataframe, column):
    return dataframe[column].value_counts()

In [1159]:
val_count(df,'Source')

Twitter                818
India Posts English     89
The Times of India      32
Cartoq                  21
Todays Chronic          14
                      ... 
TechCodex                1
Prameyanews              1
Orissa Diary             1
ODISHABARTA.COM          1
NFAPost                  1
Name: Source, Length: 230, dtype: int64

In [1160]:
print(val_count(df,'Country'))

# Will delete this column as the only country in the dataset is India
df.drop(columns='Country', inplace=True)

India    1528
Name: Country, dtype: int64


--------------------------------------------------------------------------------------------------------------------------------------------------------

In [1161]:
val_count(df,'Subregion')

Mumbai            87
Uttar Pradesh     75
Delhi             52
Madhya Pradesh    21
National          18
Maharashtra       14
West Bengal       10
Haryana            8
Kerala             7
Karnataka          5
Rajasthan          4
Orissa             4
Chennai            4
Gujarat            3
J&K                2
Punjab             2
Jharkhand          1
Chattisgarh        1
Kolkata            1
Uttrakhand         1
Bangalore          1
Name: Subregion, dtype: int64

In [1162]:
val_count(df,'State')

Tamil Nadu           264
Maharashtra          218
Delhi                 96
Uttar Pradesh         95
Karnataka             71
Kerala                43
Gujarat               35
Telangana             33
Madhya Pradesh        28
Rajasthan             25
West Bengal           21
Haryana               21
Puducherry            13
Odisha                11
Andhra Pradesh        10
Chandigarh             9
Jharkhand              8
Punjab                 5
Bihar                  5
Jammu and Kashmir      4
Chhattisgarh           4
Assam                  2
Uttarakhand            2
Arunachal Pradesh      1
Tripura                1
Goa                    1
Mizoram                1
Himachal Pradesh       1
Name: State, dtype: int64

In [1163]:
''' The variables state and subregion are supposed to contain specific & seperate type of data but if we look closely then most of these values in both of
these columns contain information related to the state from where the source is from, so lets fill all the null values in these two columns by 
refering them to each other'''

print('State Column null values (count):',df['State'].isna().sum())
print('Subregion Column null values (count):',df['Subregion'].isna().sum())

df['Subregion']=df['Subregion'].fillna(df['State'])
df['State']=df['State'].fillna(df['Subregion'])

print('New Count State:', df['State'].isna().sum())
print('New Count Subregion:', df['Subregion'].isna().sum())

State Column null values (count): 500
Subregion Column null values (count): 1207
New Count State: 497
New Count Subregion: 497


In [1164]:
val_count(df,'Subregion')

Tamil Nadu           248
Maharashtra          125
Delhi                102
Uttar Pradesh         95
Mumbai                87
Karnataka             69
Kerala                43
Gujarat               34
Telangana             33
Madhya Pradesh        28
Rajasthan             24
Haryana               21
West Bengal           20
National              18
Puducherry            13
Andhra Pradesh        10
Chandigarh             9
Jharkhand              8
Odisha                 7
Punjab                 5
Bihar                  5
Orissa                 4
Chennai                4
Chhattisgarh           3
Assam                  2
Jammu and Kashmir      2
J&K                    2
Arunachal Pradesh      1
Uttrakhand             1
Uttarakhand            1
Tripura                1
Goa                    1
Chattisgarh            1
Mizoram                1
Himachal Pradesh       1
Kolkata                1
Bangalore              1
Name: Subregion, dtype: int64

In [1165]:
val_count(df,'State')

Tamil Nadu           264
Maharashtra          218
Delhi                 96
Uttar Pradesh         95
Karnataka             71
Kerala                43
Gujarat               35
Telangana             33
Madhya Pradesh        28
Rajasthan             25
West Bengal           21
Haryana               21
Puducherry            13
Odisha                11
Andhra Pradesh        10
Chandigarh             9
Jharkhand              8
Punjab                 5
Bihar                  5
Jammu and Kashmir      4
Chhattisgarh           4
National               3
Assam                  2
Uttarakhand            2
Arunachal Pradesh      1
Tripura                1
Goa                    1
Mizoram                1
Himachal Pradesh       1
Name: State, dtype: int64

In [1166]:
# The most ideal thing to do will be to remove the sub region column
df.drop(columns='Subregion', inplace=True)

--------------------------------------------------------------------------------------------------------------------------------------------------------

In [1167]:
val_count(df, 'Language')

English    1507
Hindi        14
Bengali       6
Marathi       1
Name: Language, dtype: int64

In [1168]:
val_count(df,'Sentiment')

Neutral     850
Positive    420
Negative    258
Name: Sentiment, dtype: int64

--------------------------------------------------------------------------------------------------------------------------------------------------------

In [1169]:
val_count(df,'Keywords')

cars,Tata                 270
Cars,Tata                 267
Hyundai,cars              255
Tata,cars                 242
cars,Hyundai              152
                         ... 
Hyundai,cars,Tata,Cars      1
cars,Tata,TATA,Hyundai      1
cars,Hyundai,Cars,Tata      1
Hyundai,Cars,hyundai        1
Cars,Tata,cars,Hyundai      1
Name: Keywords, Length: 65, dtype: int64

In [1170]:
df['Keywords'].unique()

array(['Hyundai,Cars', 'Tata,cars', 'cars,Tata', 'Hyundai,Cars,hyundai',
       'Hyundai,cars', 'cars,Hyundai', 'Cars,Tata', 'CARS,Hyundai',
       'Hyundai,cars,Cars', 'cars,Tata,Hyundai', 'Cars,Hyundai,Tata',
       'tata,cars', 'Cars,Tata,Hyundai', 'cars,tata', 'Tata,Cars,tata',
       'Cars,cars,Tata', 'TATA,Cars', 'Tata,Cars', 'Cars,Hyundai',
       'TATA,cars', 'cars,Hyundai,Cars', 'hyundai,cars',
       'cars,Hyundai,Cars,Tata', 'Tata,cars,Hyundai', 'TATA,CARS',
       'cars,Hyundai,Tata', 'Tata,cars,Cars', 'cars,Tata,TATA,Hyundai',
       'cars,TATA', 'Hyundai,cars,Tata,Cars', 'CARS,Hyundai,cars',
       'tata,Tata,cars', 'Tata,Cars,tata,cars', 'Hyundai,Cars,cars',
       'cars,hyundai,tata', 'Hyundai,cars,Tata', 'Hyundai,Tata,cars',
       'Cars,cars,Tata,tata', 'Tata,Cars,cars', 'cars,Tata,tata',
       'Hyundai,cars,hyundai', 'cars,hyundai', 'cars,Cars,Tata',
       'cars,Hyundai,tata', 'cars,Tata,Hyundai,Cars', 'Cars,HYUNDAI',
       'Tata,tata,cars', 'cars,Tata,Cars', 'car

In [1171]:
df['Keywords']=df['Keywords'].str.lower()
df['Keywords'].unique()

array(['hyundai,cars', 'tata,cars', 'cars,tata', 'hyundai,cars,hyundai',
       'cars,hyundai', 'hyundai,cars,cars', 'cars,tata,hyundai',
       'cars,hyundai,tata', 'tata,cars,tata', 'cars,cars,tata',
       'cars,hyundai,cars', 'cars,hyundai,cars,tata', 'tata,cars,hyundai',
       'tata,cars,cars', 'cars,tata,tata,hyundai',
       'hyundai,cars,tata,cars', 'tata,tata,cars', 'tata,cars,tata,cars',
       'hyundai,cars,tata', 'hyundai,tata,cars', 'cars,cars,tata,tata',
       'cars,tata,tata', 'cars,tata,hyundai,cars', 'cars,tata,cars',
       'tata,cars,hyundai,cars', 'tata,hyundai,cars',
       'cars,hyundai,hyundai,cars', 'cars,tata,cars,hyundai'],
      dtype=object)

In [1172]:
# To reduce the data redundancy, I will set 3 categories tata, hyundai, tata_hyundai

brand_keywords = {
    'hyundai,cars':'hyundai', 'tata,cars':'tata', 'cars,tata':'tata', 'hyundai,cars,hyundai':'hyundai',
       'cars,hyundai':'hyundai', 'hyundai,cars,cars':'hyundai', 'cars,tata,hyundai':'tata_hyundai',
       'cars,hyundai,tata':'tata_hyundai', 'tata,cars,tata':'tata', 'cars,cars,tata':'tata',
       'cars,hyundai,cars':'hyundai', 'cars,hyundai,cars,tata':'tata_hyundai', 'tata,cars,hyundai':'tata_hyundai',
       'tata,cars,cars':'tata', 'cars,tata,tata,hyundai':'tata_hyundai',
       'hyundai,cars,tata,cars':'tata_hyundai', 'tata,tata,cars':'tata', 'tata,cars,tata,cars':'tata',
       'hyundai,cars,tata':'tata_hyundai', 'hyundai,tata,cars':'tata_hyundai', 'cars,cars,tata,tata':'tata',
       'cars,tata,tata':'tata', 'cars,tata,hyundai,cars':'tata_hyundai', 'cars,tata,cars':'tata',
       'tata,cars,hyundai,cars':'tata_hyundai', 'tata,hyundai,cars':'tata_hyundai',
       'cars,hyundai,hyundai,cars':'hyundai', 'cars,tata,cars,hyundai':'tata_hyundai'
}

df['Keywords']=df['Keywords'].map(brand_keywords)

In [1173]:
df['Keywords'].value_counts()

tata            984
hyundai         477
tata_hyundai     67
Name: Keywords, dtype: int64

--------------------------------------------------------------------------------------------------------------------------------------------------------

In [1174]:
df.isna().sum()

Date                       0
Headline                 818
URL                        0
Opening Text             818
Hit Sentence               0
Source                     0
Influencer               463
Language                   0
Reach                      9
Desktop Reach            818
Mobile Reach             818
Twitter Social Echo      818
Facebook Social Echo     818
Reddit Social Echo       818
National Viewership        0
Engagement              1176
Sentiment                  0
Keywords                   0
State                    497
dtype: int64

In [1175]:
print(df['National Viewership'].value_counts())
# Need to remove this variable 
df.drop(columns='National Viewership', inplace=True)

0    1528
Name: National Viewership, dtype: int64


In [1176]:
df['Reach'].isna().sum() # It will make sense to remove these rows as there will not be much data loss

df.dropna(subset='Reach', inplace=True)

In [1177]:
# Will remove all the unnecessary columns with high null values
df.drop(columns=['Desktop Reach', 'Mobile Reach', 'Twitter Social Echo', 'Reddit Social Echo', 'Facebook Social Echo', 'Opening Text', 'Headline'], inplace=True)

In [1178]:
df.isna().sum()

Date               0
URL                0
Hit Sentence       0
Source             0
Influencer       463
Language           0
Reach              0
Engagement      1170
Sentiment          0
Keywords           0
State            494
dtype: int64

In [1179]:
print(df.Influencer.value_counts())
# Looking at the value count of Influencer, we can remove this column as well.
df.drop(columns='Influencer', inplace=True)

@jayeshkhunt20      21
@CSAlsundkar        20
@MaheshKhokle       18
@mr_singh1888       13
Utkarsh Deshmukh    12
                    ..
@_autocrat           1
@jayanth_s_iyer      1
@thoughtsofSuraJ     1
@mstock_in           1
@poojashah0998       1
Name: Influencer, Length: 761, dtype: int64


------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1180]:
df['State'].isna().sum()

''' State column is important because it can provide a lot of insights. 
Therefore we can assign the null values with the value count ratio of the rest of the data
The data has a total of 1519 observations and the null vales are 494, it is better to impute 
the missing values with the column names on the basis of value count ratios of remaining values'''

' State column is important because it can provide a lot of insights. \nTherefore we can assign the null values with the value count ratio of the rest of the data\nThe data has a total of 1519 observations and the null vales are 494, it is better to impute \nthe missing values with the column names on the basis of value count ratios of remaining values'

In [1181]:
# Defining the population ratios
population_ratios = {
    "Tamil Nadu": 0.3255,
    "Maharashtra": 0.2693,
    "Delhi": 0.1188,
    "Uttar Pradesh": 0.1176,
    "Karnataka": 0.0879,
    "Kerala": 0.0532,
    "Gujarat": 0.0433,
    "Telangana": 0.0396,
    "Madhya Pradesh": 0.0347,
    "Rajasthan": 0.0272,
    "West Bengal": 0.0260,
    "Haryana": 0.0260,
    "Puducherry": 0.0149,
    "Odisha": 0.0136,
    "Andhra Pradesh": 0.0124,
    "Chandigarh": 0.0111,
    "Jharkhand": 0.0099,
    "Punjab": 0.0062,
    "Bihar": 0.0062,
    "Jammu and Kashmir": 0.0049,
    "Chhattisgarh": 0.0049,
    "National": 0.0037,
    "Assam": 0.0025,
    "Uttarakhand": 0.0025,
    "Arunachal Pradesh": 0.0012,
    "Tripura": 0.0012,
    "Goa": 0.0012,
    "Mizoram": 0.0012,
    "Himachal Pradesh": 0.0012
}

# Function to impute null values in "state" column based on ratios
def impute_state(row):
    if pd.isna(row["State"]):
        # Generate a random value based on the population ratios
        state = random.choices(list(population_ratios.keys()), list(population_ratios.values()))[0]
        return state
    else:
        return row["State"]

# Apply the impute_state function to fill null values in the "state" column
df["State"] = df.apply(impute_state, axis=1)

------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1189]:
df.isna().sum()

# Considered & explored the option to impute all missing values through regression, 
# but it is not the best approach as most of the values in the  
# engagement column are null (1170)

# Also making assumptions and imputing the missing values with a specific number or ratio might not be the best practices,
# so the best option is to remove the column

Date               0
URL                0
Hit Sentence       0
Source             0
Language           0
Reach              0
Engagement      1170
Sentiment          0
Keywords           0
State              0
dtype: int64

In [1190]:
df.drop(columns='Engagement', inplace=True)

------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Feature Engineering - Creating sentiment column on the basis of Hit Sentence

In [1196]:
# Converting Hit Scentence to lower case
df['Hit Sentence']=df['Hit Sentence'].str.lower()

df

,Date,URL,Hit Sentence,Source,Language,Reach,Sentiment,Keywords,State
0,28-Sep-2023 02:08PM,https://hindi.pardaphash.com/kia-and-hyundai-r...,kia and hyundai recalled cars: वाहन निर्माता क...,Pardaphash,Hindi,31307.0,Neutral,hyundai,Delhi
1,28-Sep-2023 01:46PM,https://economictimes.indiatimes.com/industry/...,... punch ev is expected to compete with the m...,The Economic Times,English,15403059.0,Neutral,tata,Maharashtra
2,28-Sep-2023 01:30PM,https://badisoch.in/automobile-news/tata-aviny...,"... , despite most carmakers focusing on large...",Badi Soch,English,1128280.0,Neutral,tata,Karnataka
3,28-Sep-2023 01:24PM,http://twitter.com/automobilindia8/statuses/17...,engine fire-related risks force kia & hyundai ...,Twitter,English,2.0,Neutral,hyundai,Karnataka
4,28-Sep-2023 01:22PM,http://twitter.com/MalakpetD/statuses/17073021...,"@srinualavilli @bmtc_bengaluru mahindra, tata,...",Twitter,English,506.0,Neutral,tata,Telangana
...,...,...,...,...,...,...,...,...,...
1523,12-Sep-2023 06:20AM,https://bulletinreporter.com/mg3-hatchback-tak...,"... costs survey, which lists the cost of runn...",Bulletin Reporter,English,2074.0,Positive,hyundai,Gujarat
1524,12-Sep-2023 05:24AM,https://vervetimes.com/mg3-hatchback-named-by-...,"... costs survey, which lists the cost of runn...",Verve times - Latest News around the World,English,26425.0,Positive,hyundai,Maharashtra
1525,12-Sep-2023 04:37AM,https://newsconcerns.com/racv-names-mg3-hatchb...,"... costs survey, which lists the cost of runn...",News Concerns - Latest News Update,English,9323.0,Positive,hyundai,Madhya Pradesh
1526,12-Sep-2023 04:04AM,https://thenfapost.com/tata-power-ez-app-elect...,charging hubs for leading cab fleet operators....,NFAPost,English,851.0,Neutral,tata,Karnataka
